# Метод ближайшего соседа (1NN) с евклидовой метрикой для задачи классификации

**Генерация данных** для демонстрации работы метода.

In [1]:
%matplotlib inline

import matplotlib
matplotlib.style.use('ggplot')

from matplotlib import pyplot
from matplotlib.colors import ListedColormap

from sklearn import datasets, ensemble, tree, model_selection, metrics
import xgboost as xgb

import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [126]:
"""Выборка с целыми неотрицательными значениями признаков (1797, 65)"""

# Загрузка встроенного "игрушечного" набора данных digits
digits = datasets.load_digits()

In [127]:
# Разделение данных на обучающую и тестовую выборки

# Определеим 25% выборки на тест
size = int(len(data)*0.25)

# Обучающая выборка
X_train = digits.data[:-size, :]
y_train = digits.target[:-size]

# Тестовая выборка
X_test = digits.data[-size:, :]
y_test = digits.target[-size:]

In [128]:
print(f'Обучающая выборка:\tX={X_train.shape} \t\tТестовая выборка:\tX={X_test.shape}')
print(f'\t\t\ty={y_train.shape} \t\t\t\t\ty={y_test.shape}')

Обучающая выборка:	X=(1348, 64) 		Тестовая выборка:	X=(449, 64)
			y=(1348,) 					y=(449,)


**Метод ближайшего соседа (1NN)** - самый простой метрический метод в задаче классификации. Суть метода заключается в том, что новая точка относится к такому же классу, что и ближайшая к ней точка из обучающей выборки.
<img src="1NN.png" width=350 height=350>

Метрика является функцией, задающей расстояние в метрическом пространстве, и должна удовлетворять следующим аксиомам:
1. $\rho\left(x,\ y\right)\geq0$, причем $\rho\left(x,\ y\right)=0\ \ \Longleftrightarrow\ \ x=y$.
2. $\rho\left(x,\ y\right)=\rho\left(y,\ x\right)$,
3. $\rho\left(x,\ y\right)\le\rho\left(x,\ z\right)+\rho\left(z,\ y\right)$.

В данном примере будет реализована __Евклидова метрика__:

$$\rho\left(x,\ y\right)=\sqrt{\sum_{i=1}^{n}\left(x_i-y_i\right)^2}$$

In [156]:
def classifer_1NN(X):
    """Функция принимает вектор признаков объекта и возвращает оценку класса объекта, которому он соотвествует,
    полученную по обучающей выборке"""
    # Расстояние от объекта X до каждого из обучающей выборки в n-мерном пространстве по Евклидовой метрике:
    # Поскольку корень из суммы квадратов отклонений — монотонное преобразование и не влияет на результат работы 
    # алгоритма, можно его не извлекать.
    d = [np.sum((X - X_train[i,:])**2) for i in range(X_train.shape[0])]
    
    # Индекс ближайшего оьбъекта
    ind = d.index(min(d))

    # Соответствующий класс ближайшего объекта
    return(y_train[ind])

In [158]:
%%time
# Предсказание ответов на отложенном тесте
predictions = [classifer_1NN(X = X_test[i,:]) for i in range(X_test.shape[0])]

Wall time: 7.49 s


In [163]:
# Функция accuracy_score принимает метки тестовой выборки и предсказанные метки predictions
1- metrics.accuracy_score(y_test, predictions)

0.03786191536748329

## Сравнение метода ближайшего соседа (1NN) и случайного леса

In [160]:
# Модель случайного леса с 1000-ю деревьями, каждое глубиной не более 2
rf_classifier = ensemble.RandomForestClassifier(n_estimators = 1000)
rf_classifier.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [161]:
rf_predictions = rf_classifier.predict(X_test)

In [164]:
# Функция accuracy_score принимает метки тестовой выборки и предсказанные метки predictions
1 - metrics.accuracy_score(y_test, rf_predictions)

0.06904231625835189